# PageRank
Today we will implement PageRank algorithm for a small collection of document about Information Retrieval. For this we will extract link information from Wikipedia and build a Google Matrix. 

Based on the matrix you will build a *reputation ranking for the documents*.

<!-- To obtain ranking result you can use:
- Naïve approach with matrix inversion
- Power Method
 -->
# [OPTIONAL] Download a dataset

Please avoid submitting your solutions with `np.linalg.eig`, even you can use it to verify you result.

# 1. You may start the lab from this place
Use the code below in your solution to load the adjacency matrix.

In [9]:
import numpy as np
import plotly.express as px

In [10]:
pages = [ "Bag-of-words model", "Bayes' theorem", "Cluster analysis", "Content-based image retrieval", "Database", "Deep learning", "Desktop search", "Dimensionality reduction", "Discounted Cumulative Gain", "Eigenvector", "Full-text search", "Hypertext", "Image retrieval", "Information retrieval", "Information system", "K-nearest neighbors algorithm", "Language model", "Latent Dirichlet allocation", "Latent semantic analysis", "Low-rank approximation", "Multimedia information retrieval", "Netflix Prize", "Netflix", "Ranking (information retrieval)", "Recommender systems", "Relevance (information retrieval)", "Rocchio algorithm", "Search algorithm", "Search engines", "Semantic search", "Semantic web", "Sentiment analysis", "Similarity search", "Site search", "Text mining", "Text Retrieval Conference", "Tf–idf", "Vector space model", "Web crawler", "World Wide Web"]
Atext = "0000000000000000011000000000000100100100000000000000010000000000000000000000000000000101000000000000100010000000000000000000001000001000000010000000000000000000000110000001111000000000000111100010000100100000000000000010000000000001000000000000000000001100000000000000110001000010000001000100000000000100100000001010000000000000000000000000000010000000000000000000000000000000001000000000000000000000000000000000010000000000000000000000000000001000000001000000000000000110010000010001000000000100000010000000110001000010101010001010101011101001111010100111110000001000000001000000000000000000000000010000010100000000000010001000000000000000100001000000010000000000000000000000000010000000000001000010000000000001001010001000000100000100010100000100001110101100000000000000000000100000000000000000000000010000000011000000000000000000000000000000000100000000000000101000000010000000000000000000000000000100100000000000000000000000000001000000000000010000000000000010000000000100000000000000000000000000000000001010010000000001001010000010110000000000000000000000000010000000000000000000000000000000000000001000000000000000000000000000011000000000000000000000001100001000000110000000000000001010010000110000000000001000000000000000010000100000100001000000000001100000100000000010000000000001000000000000010010000000000000000000000000000100000000000000000000000000100000000000100001100000100011010100101000000000000001000000000001000000000000001000000000000100001000001000000000000100100000000000010000101000001000000000100000000000000010000000000000001110010000010000100010010110000000000000111000000010"

def load(text, w=40):
    return np.array([float(a) for a in text]).reshape((w, -1))

A = load(Atext)

px.imshow(A)

## 1.1. [TASK] Prepare a stochastic matrix M based on adjacency matrix A

Write the code which norms matrix A column-wise. Add $\frac{1}{N}$ factor where outdegree is 0.

You can refer to wikipedia's [Google Matrix](https://en.wikipedia.org/wiki/Google_matrix#Adjacency_matrix_A_and_Markov_matrix_S) article. In construction algorthm this matrix is referred as `Markov matrix S`.

In [38]:
M = A.copy()

for j in range(0, M.shape[1]):
    k = np.sum(A[:, j])
    if k == 0:
        M[:, j] = 1 / M.shape[1]
    else:
        M[:, j] = A[:, j] / k


In [39]:
px.imshow(M)

## 1.2. [TASK] Prepare the Google matrix

Compute the Google matrix as described in construction block of [Google Matrix](https://en.wikipedia.org/wiki/Google_matrix#Construction_of_Google_matrix_G) article. 

`S` there is our matrix $\mathcal{M}$.

$\alpha$ is a damping factor, which is accepted to be exactly `0.85`.

In [40]:
def to_google(M, alpha=0.85):
    return alpha * M + (1 - alpha) * 1 / M.shape[1]

In [41]:
G = to_google(M)

In [42]:
px.imshow(G)

## 1.3. [TASK] Solve naively

Everything is ready for solution! Obvious way -- is to use algeraic solution of the equation.

$\mathbf{R} = d \mathcal{M}\mathbf{R} + \frac{1-d}{N} \mathbb{1}$

Remember the hack from the lecture, that:

$\mathbb{E}\times \mathbf{R}=\mathbb{1}$

In [31]:
alpha = 0.85
F = 1 - alpha * M

In [35]:
R = np.linalg.inv(np.identity(M.shape[1]) - alpha * M) @ ((1 - alpha) / M.shape[1] * np.ones((M.shape[1], 1)))

print(np.argsort(R.reshape(-1)))

[26  8 27 33 10  1 19  9 23 22 24 16 30  2 15 35 32  5 20 36  6 38 28 14
 37 31  0 21 17  3 12 11  7 25 34 29 39 18  4 13]


## 1.4. [TASK] Solve with power method

You can also use [Power method](https://en.wikipedia.org/wiki/Power_iteration) to obtain dominating eigenvector.
$R = G^{N}v_{random}$

In [59]:
v = np.random.random((M.shape[1], 1))
v = v / np.linalg.norm(v, 1)

v = np.linalg.matrix_power(G, 128) @ v

assert np.allclose(v, R)
print(np.argsort(v.reshape(-1)))

[26 27  8  1 33 10  9 19 23 22 24 16 30  2 15 35 32  5 20 36  6 38 28 14
 37 31  0 21 17  3 12 11  7 25 34 29 39 18  4 13]


## 1.5. Built-in check

This code below allows you to check your solution, but we do not accept it as a solution.

In [60]:
evals, evecs = np.linalg.eig(G)
print(np.argsort(evecs[:, 0]))

[26 27  8 10 33  1  9 19 23 22 24 16 30 15  2 35 32  5 20 36  6 38 28 14
 37 31  0 21 17  3 12 11  7 25 34 29 39 18  4 13]


In [57]:
#print(np.argsort(evecs[:, 0]) == np.argsort(v))

# 2. [TASK] Ranking

Print the ranking. First should come the documents with *the highest* PageRank.

In [65]:
for b in np.argsort(v.reshape(-1))[::-1]:
    print(f"{b:2} {pages[b]}")

13 Information retrieval
 4 Database
18 Latent semantic analysis
39 World Wide Web
29 Semantic search
34 Text mining
25 Relevance (information retrieval)
 7 Dimensionality reduction
11 Hypertext
12 Image retrieval
 3 Content-based image retrieval
17 Latent Dirichlet allocation
21 Netflix Prize
 0 Bag-of-words model
31 Sentiment analysis
37 Vector space model
14 Information system
28 Search engines
38 Web crawler
 6 Desktop search
36 Tf–idf
20 Multimedia information retrieval
 5 Deep learning
32 Similarity search
35 Text Retrieval Conference
15 K-nearest neighbors algorithm
 2 Cluster analysis
30 Semantic web
16 Language model
24 Recommender systems
22 Netflix
23 Ranking (information retrieval)
19 Low-rank approximation
 9 Eigenvector
10 Full-text search
33 Site search
 1 Bayes' theorem
 8 Discounted Cumulative Gain
27 Search algorithm
26 Rocchio algorithm
